# JCF cell analysis

## Import packages

If you want modules to be automatically reloaded when you call them, use `autoreload`:

In [ ]:
%load_ext autoreload
%autoreload 2

Import packages

In [ ]:
import anndata
import scanpy as sc
import os
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cellrank as cr
import scvelo as scv

In [ ]:
scv.set_figure_params()
sc.settings.set_figure_params(dpi=100, facecolor='white', frameon=False)

In [ ]:
from utils import *

In [ ]:
my_cmap = get_continuous_cmap(['D6D6D6', '5D4FF4'])

## Print package versions for reproducibility

If you want to exactly reproduce the results shown here, please make sure that your package versions match what is printed below. 

In [ ]:
cr.logging.print_versions()

In [ ]:
cr.logging.print_version_and_date()

## Setup paths

In [ ]:
data_path = '/lustre/groups/ml01/workspace/laura.martens/moretti_colab'
save_path = os.path.join(data_path, 'scglue')
scvelo_path = os.path.join(data_path, 'scvelo')

In [ ]:
fig_path = '/lustre/groups/ml01/workspace/laura.martens/moretti_colab/panels/jcf'
sc.settings.figdir = fig_path

# Load data

In [ ]:
adata = sc.read('/lustre/groups/ml01/workspace/laura.martens/moretti_colab/scglue/metacells_imputed.h5ad')

In [ ]:
# Load scvelo results
scvelo_adata = sc.read(os.path.join(scvelo_path, 'scvelo_adata_non_dl.h5ad'))

In [ ]:
scvelo_adata = scvelo_adata[adata.obs_names].copy()

In [ ]:
scvelo_adata.obsm['X_umap'] = adata.obsm['X_umap']
scvelo_adata.obsm['X_glue'] = adata.obsm['X_glue']
scvelo_adata.obs = scvelo_adata.obs.join(adata.obs.leiden_res1, lsuffix='old')

In [ ]:
scvelo_adata.uns.update({'leiden_res1_colors': adata.uns['leiden_res1_colors']})

## Load marker genes

In [ ]:
markers = pd.read_excel(os.path.join(data_path, 'transfer_data', 'Markers_lineages.xlsx'))

In [ ]:
markers = {key: markers[key][~markers[key].isna()].to_list() for key in markers.columns}

## Load cellrank results

In [ ]:
from cellrank.tl.kernels import PseudotimeKernel, ConnectivityKernel, VelocityKernel
from cellrank.tl.estimators import GPCCA

In [ ]:
terminal_states = ['Epicardial', 'Cardiomyocyte']

In [ ]:
g = GPCCA.read(os.path.join(data_path, 'cellrank', 'g.pickle'))

In [ ]:
g.adata = adata

In [ ]:
g.compute_absorption_probabilities(solver='gmres', use_petsc=True)

# Restrict to Pre-JCF, JCF and Epicardial

In [ ]:
cluster = ['14']
adata_epi = g.adata[g.adata.obs.leiden_res1.isin(cluster)].copy()

In [ ]:
adata_epi_scvelo = scvelo_adata[adata_epi.obs_names]

## Plot UMAP

In [ ]:
for key in ['leiden_res1', 'atac_match', 'day']:
    fig, ax = plt.subplots(figsize=(6, 2))
    sc.pl.umap(adata_epi, color=[key], save=f'jcf_{key}.pdf', ax=ax)

In [ ]:
adata_epi.obs['Epicardial'] = adata_epi.obsm['to_terminal_states']["Epicardial"].X

In [ ]:
adata_epi.obs['Endoderm'] = adata_epi.obsm['to_terminal_states']["Endoderm"].X

In [ ]:
adata_epi.obs['Cardiomyocyte'] = adata_epi.obsm['to_terminal_states']['Cardiomyocyte'].X

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
sc.pl.umap(adata_epi, color=['Cardiomyocyte'], save='ap_jcf_cardiomyocytes.pdf',  vmin=[0.2], ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
sc.pl.umap(adata_epi, color=['Epicardial'], save='ap_jcf_epicardial.pdf', ax=ax)

In [ ]:
# test z scored absorption probability
adata_epi.obs['Epicardial_zscore'] = scipy.stats.zscore(adata_epi.obsm['to_terminal_states']["Epicardial"].X)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
sc.pl.umap(adata_epi, color=['Epicardial_zscore'], save='ap_jcf_epicardial_zscore.pdf', ax=ax)

## Kmeans clustering

In [ ]:
from sklearn.cluster import KMeans
# extract pca coordinates
absorption_prob = adata_epi.obs.loc[:, ['Cardiomyocyte']].values.reshape(-1, 1)

# kmeans with k=5
kmeans = KMeans(n_clusters=2, random_state=0).fit(absorption_prob) 
adata_epi.obs['kmeans'] = kmeans.labels_.astype(str)
adata_epi.obs['cluster'] = adata_epi.obs['kmeans'].map({'0': 'CM', '1': 'Epi'})
fig, ax = plt.subplots(figsize=(6, 2))
sc.pl.umap(adata_epi, color=['cluster'], save='ap_jcf_cluster.pdf',ax=ax, palette=['#023fa5', '#ef9708'])

## Save absorption probabilities

In [ ]:
prob = adata_epi.obs.loc[:, ['Epicardial', 'Cardiomyocyte', 'Epicardial_zscore', 'cluster']]

In [ ]:
prob.to_csv('/lustre/groups/ml01/workspace/laura.martens/moretti_colab/pando/absorption_prob.csv')

## Plot violin

In [ ]:
ax = sc.pl.violin(adata_epi, keys='Epicardial', groupby='leiden_res1', show=False)
plt.ylabel('Absorption probability\nEpicardial lineage')
plt.savefig(os.path.join(fig_path, 'violin_jcf__epicardial_ap.pdf'))

In [ ]:
ax = sc.pl.violin(adata_epi, keys='Cardiomyocyte', groupby='leiden_res1', show=False)
plt.ylabel('Absorption probability\nCardiomyocyte lineage')
plt.savefig(os.path.join(fig_path, 'violin_jcf_cardiomyocyte_ap.pdf'))

# Plot module scores

In [ ]:
module_score_pos = pd.read_csv(os.path.join(fig_path, 'module_score_pos.csv'), index_col=0)

In [ ]:
module_pos = ad.AnnData(module_score_pos, obs=adata_epi.obs, uns=adata_epi.uns, obsm=adata_epi.obsm)

In [ ]:
module_score_neg = pd.read_csv(os.path.join(fig_path, 'module_score_neg.csv'), index_col=0)

In [ ]:
module_neg = ad.AnnData(module_score_neg, obs=adata_epi.obs, uns=adata_epi.uns, obsm=adata_epi.obsm)

In [ ]:
features = pd.Series(['HAND2', 'FOS', 'EPAS1', "MSX1", "ISL1", "HMGB3", "HEY1", "HOXB1", "MEIS1", "GATA4"])

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(6, 2))
    sc.pl.umap(module_pos, color=feature, vmin="p01", vmax="p99", ax=ax, save=f"pos_module_score_{feature}.pdf")

In [ ]:
for feature in features[features.isin(module_neg.var_names)]:
    fig, ax = plt.subplots(figsize=(6, 2))
    sc.pl.umap(module_neg, color=feature, vmin="p01", vmax="p99", ax=ax, save=f"neg_module_score_{feature}.pdf")

## Plot expression of downstream targets

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
gene = 'TFAP2B'
sc.pl.umap(adata_epi, color=gene, vmin="p01", vmax="p99", save=f"{gene}.pdf", cmap=my_cmap, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
gene = 'GATA4'
sc.pl.umap(adata_epi, color=gene, vmin="p01", vmax="p99", save=f"{gene}.pdf", cmap=my_cmap, ax=ax)

In [ ]:
targets = ['DPF3', 'NEBL', 'RP11-332H18.4']

In [ ]:
for feature in targets:
    fig, ax = plt.subplots(figsize=(6, 2))
    sc.pl.umap(adata_epi, color=feature, vmin="p01", vmax="p99", ax=ax, save=f"pos_gata4_target_{feature}.pdf", cmap=my_cmap)

In [ ]:
sc.set_figure_params(figsize=(6,2), frameon=False)
sc.pl.umap(adata_epi, color=targets, vmin="p01", vmax="p99", save=f"pos_gata4_target_all.pdf", cmap=my_cmap)

In [ ]:
sc.set_figure_params(figsize=(6,2), frameon=False)
sc.pl.umap(adata_epi, color=['LRP2'], vmin="p01", vmax="p99", save=f"neg_gata4_target_all.pdf", cmap=my_cmap)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
sc.pl.umap(adata_epi, color="LRP2", vmin="p01", vmax="p99", ax=ax, save=f"neg_gata4_target_LRP2.pdf", cmap=my_cmap)

In [ ]:
targets = ['AHNAK' , 'CDC42EP1', 'CGN' , 'ELMOD1' , 'EPAS1' , 'F11R' , 'FUT8' , 'GPC5' , 'KCNMA1' , 'KRT19' , 'MXRA8' , 'PDLIM1' , 'PRSS23' , 'RIMS2' , 'SLC28A2' , 'SPINT2' , 'SWAP70' , 'TGIF1' , 'VTCN1' , 'WNT6' , 'WNT7B']

In [ ]:
for feature in targets:
    fig, ax = plt.subplots(figsize=(6, 2))
    sc.pl.umap(adata_epi, color=feature, vmin="p01", vmax="p99", ax=ax, save=f"pos_tfap2b_target_{feature}.pdf", cmap=my_cmap)

In [ ]:
sc.set_figure_params(figsize=(6,2), frameon=False)
sc.pl.umap(adata_epi, color=targets, vmin="p01", vmax="p99", save=f"pos_tfap2b_target_all.pdf", cmap=my_cmap)

In [ ]:
for feature in ['METRN']:
    fig, ax = plt.subplots(figsize=(6, 2))
    sc.pl.umap(adata_epi, color=feature, vmin="p01", vmax="p99", save=f"neg_tfap2b_target_{feature}.pdf", cmap=my_cmap, ax=ax)

In [ ]:
sc.pl.umap(adata_epi, color=['METRN'], vmin="p01", vmax="p99", save=f"neg_tfap2b_target_all.pdf", cmap=my_cmap)

In [ ]:
g.adata = adata

In [ ]:
cluster =['14']

In [ ]:
dfs = {}
for lineage in terminal_states:
    print(lineage)
    df = g.compute_lineage_drivers(lineages=[lineage], layer='log_counts', cluster_key='leiden_res1', clusters=cluster, method='perm_test', n_jobs=-1)
    dfs.update({lineage: df})

In [ ]:
epi_drivers = dfs['Epicardial'].dropna()
epi_drivers = epi_drivers[epi_drivers['Epicardial_qval'] < 0.05].sort_values('Epicardial_corr', ascending=False)

In [ ]:
cm_drivers = dfs['Cardiomyocyte'].dropna()
cm_drivers = cm_drivers[cm_drivers['Cardiomyocyte_qval'] < 0.05].sort_values('Cardiomyocyte_corr', ascending=False)

In [ ]:
gene_drivers = {'Epicardial': epi_drivers, 'Cardiomyocyte': cm_drivers}

In [ ]:
for lineage in terminal_states:
    print(lineage)
    genes = gene_drivers[lineage].index
    sc.pl.umap(adata_epi, color=genes[:40], save=f"gene_lineage_drivers_{lineage}_{'_'.join(cluster)}.pdf", layer='log_counts', ncols=8, cmap=my_cmap, vmax='p99') #, vmax='p95', vmin='p05',

In [ ]:
epi_drivers.to_csv(os.path.join(fig_path, f"jcf_epicardial_gene_drivers_{'_'.join(cluster)}.csv"))

In [ ]:
fig_path

In [ ]:
cm_drivers.to_csv(os.path.join(fig_path, f"jcf_cm_gene_drivers_{'_'.join(cluster)}.csv"))

## Plot gene activity drivers

In [ ]:
adata_act = get_act(adata, imputed=False, fill_value=0)

In [ ]:
adata_act_epi = get_act(adata_epi, imputed=True, fill_value=0)

In [ ]:
g.adata = adata_act

In [ ]:
dfs = {}
for lineage in terminal_states:
    print(lineage)
    df = g.compute_lineage_drivers(lineages=[lineage], cluster_key='leiden_res1', clusters=cluster, method='perm_test', n_jobs=-1)
    genes = df.sort_values(f'{lineage}_corr', ascending=False).index
    sc.pl.umap(adata_act_epi, color=genes[:12], save=f'gene_activity_drivers_{lineage}.pdf', cmap=my_cmap)
    dfs.update({lineage: df})

In [ ]:
epi_drivers_act = dfs['Epicardial'].dropna()
epi_drivers_act = epi_drivers_act[epi_drivers_act['Epicardial_qval'] < 0.05].sort_values('Epicardial_corr', ascending=False)

In [ ]:
cm_drivers_act = dfs['Cardiomyocyte'].dropna()
cm_drivers_act = cm_drivers_act[cm_drivers_act['Cardiomyocyte_qval'] < 0.05].sort_values('Cardiomyocyte_corr', ascending=False)

In [ ]:
epi_drivers_act.to_csv(os.path.join(fig_path, 'jcf_epicardial_gene_activity_drivers.csv'))

In [ ]:
cm_drivers_act.to_csv(os.path.join(fig_path, 'jcf_cm_gene_activity_drivers.csv'))

## Plot motif drivers

In [ ]:
def compute_correlation(i, X, Y):
    return scipy.stats.spearmanr(X[:, i], Y)

In [ ]:
from cellrank.tl._utils import _correlation_test
from statsmodels.stats.multitest import multipletests

### Motif drivers

In [ ]:
adata_chrom_epi = get_chromvar(adata_epi, fill_value=np.nan)

In [ ]:
dfs = {}
for lineage in terminal_states:
    print(lineage)
    X = adata_chrom_epi[(adata_chrom_epi.obs.atac_match==1) & adata_chrom_epi.obs.leiden_res1.isin(cluster)].X
    Y = adata_chrom_epi[(adata_chrom_epi.obs.atac_match==1) & adata_chrom_epi.obs.leiden_res1.isin(cluster)].obsm['to_terminal_states'][lineage]
    
    df = _correlation_test(X=X, Y=Y, gene_names=adata_chrom_epi.var_names, method='perm_test', n_perms=1000, seed=1)
    #pvals = [compute_correlation(i, X, Y) for i in np.arange(X.shape[1])]
    #df = pd.DataFrame(pvals)
    #df.columns = lineage + '_' + pd.Series(['corr', 'pval'])
    #df[f"{lineage}_qval"] = multipletests(df[f"{lineage}_pval"], alpha=0.05, method="fdr_bh")[1]
    #df.index = adata_chrom_epi.var_names
    dfs.update({lineage: df})

In [ ]:
epi_drivers_motif = dfs['Epicardial'].dropna()
epi_drivers_motif = epi_drivers_motif[epi_drivers_motif['Epicardial_qval'] < 0.05].sort_values('Epicardial_corr', ascending=False)

In [ ]:
cm_drivers_motif = dfs['Cardiomyocyte'].dropna()
cm_drivers_motif = cm_drivers_motif[cm_drivers_motif['Cardiomyocyte_qval'] < 0.05].sort_values('Cardiomyocyte_corr', ascending=False)

In [ ]:
motif_drivers = {'Epicardial': epi_drivers_motif, 'Cardiomyocyte': cm_drivers_motif}

In [ ]:
for lineage in terminal_states:
    print(lineage)
    genes = motif_drivers[lineage].index[:40]
    sc.pl.umap(adata_chrom_epi, color=genes, save=f"motif_lineage_drivers_{lineage}_{'_'.join(cluster)}.pdf", vmax='p99', vmin='p01', ncols=8)
    sc.pl.umap(adata_epi, color=genes[genes.isin(adata_epi.var_names)], save=f"motif_lineage_drivers_gene_expression_{lineage}_{'_'.join(cluster)}.pdf", ncols=8, cmap=my_cmap, layer='log_counts')

In [ ]:
sc.pl.umap(adata, color='leiden_res1', legend_loc='on data')

In [ ]:
adata_cm = adata[~adata.obs.leiden_res1.isin(['2', '5', '21', '11', '13', '19', '23'])].copy()

In [ ]:
adata_chrom = get_chromvar(adata_cm, fill_value=np.nan)

In [ ]:
for lineage in terminal_states:
    print(lineage)
    genes = motif_drivers[lineage].index[:40]
    sc.pl.umap(adata_chrom, color=genes, vmax='p99', vmin='p01', ncols=8, save=f"motif_lineage_drivers_{lineage}_full.pdf")
    sc.pl.umap(adata_cm, color=genes[genes.isin(adata_epi.var_names)], ncols=8, cmap=my_cmap, layer='log_counts', save=f"motif_lineage_drivers_gene_expression_{lineage}_full.pdf")

In [ ]:
for lineage in terminal_states:
    print(lineage)
    genes = gene_drivers[lineage].index
    sc.pl.umap(adata_cm, color=genes[:40], layer='log_counts', ncols=8, cmap=my_cmap, vmax='p99', save=f"gene_lineage_drivers_{lineage}_full.pdf") #, vmax='p95', vmin='p05',

In [ ]:
epi_drivers_motif.to_csv(os.path.join(fig_path, f"jcf_epicardial_motif_drivers_{'_'.join(cluster)}.csv"))

In [ ]:
cm_drivers_motif.to_csv(os.path.join(fig_path, f"jcf_cm_motif_drivers_{'_'.join(cluster)}.csv"))

## Plot TFAP2A

In [ ]:
sc.pl.umap(adata_chrom_epi, color=['TFAP2A', 'TFAP2A(var.2)', 'TFAP2A(var.3)', 'MEIS1', 'MEIS2'], vmax='p90', vmin='p10')

In [ ]:
from sklearn.cluster import KMeans
# extract pca coordinates
absorption_prob = adata_14.obs.loc[:, ['Cardiomyocyte']].values.reshape(-1, 1)

# kmeans with k=5
kmeans = KMeans(n_clusters=2, random_state=0).fit(absorption_prob) 
adata_14.obs['kmeans'] = kmeans.labels_.astype(str)
adata_14.obs['cluster'] = adata_14.obs['kmeans'].map({'0': 'CM', '1': 'Epi'})
sc.pl.umap(adata_14, color=['cluster'])

In [ ]:
4

### Save as table

In [ ]:
adata_14_chrom = get_chromvar(adata_14, fill_value=np.nan)

In [ ]:
adata_14_act = get_act(adata_14, fill_value=0, imputed=True)

In [ ]:
#sc.tl.rank_genes_groups(adata_14, groupby='High epi', method='wilcoxon', layer='log_counts')
sc.tl.rank_genes_groups(adata_14_chrom, groupby='kmeans', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups(adata_14_chrom, n_genes=30, sharey=False)

In [ ]:
sc.tl.rank_genes_groups(adata_14, groupby='kmeans', method='wilcoxon')
sc.pl.rank_genes_groups(adata_14, n_genes=30, sharey=False)

In [ ]:
motifs = pd.DataFrame(adata_14_chrom.uns['rank_genes_groups']['names']).head(40)
genes = pd.DataFrame(adata_14.uns['rank_genes_groups']['names']).head(40)

In [ ]:
motif_drivers_de = {'Epicardial': motifs['1'].values, 'Cardiomyocyte': motifs['0'].values}
gene_drivers_de = {'Epicardial': genes['1'].values, 'Cardiomyocyte': genes['0'].values}

In [ ]:
for lineage in terminal_states:
    print(lineage)
    genes = motif_drivers_de[lineage]
    sc.pl.umap(adata_chrom_epi, color=genes, vmax='p99', vmin='p01', ncols=8)
    sc.pl.umap(adata_epi, color=genes[pd.Series(genes).isin(adata_epi.var_names)], ncols=8, cmap=my_cmap, layer='log_counts')
    sc.pl.umap(adata_chrom, color=genes, ncols=8,vmax='p99', vmin='p01')
    sc.pl.umap(adata_cm, color=genes[pd.Series(genes).isin(adata_epi.var_names)], ncols=8, cmap=my_cmap, layer='log_counts')

In [ ]:
for lineage in terminal_states:
    print(lineage)
    genes = gene_drivers[lineage]
    sc.pl.umap(adata_epi, color=genes[pd.Series(genes).isin(adata_epi.var_names)], ncols=8, cmap=my_cmap, layer='log_counts')
    sc.pl.umap(adata, color=genes[pd.Series(genes).isin(adata_epi.var_names)], ncols=8, cmap=my_cmap, layer='log_counts')

## Find overlap

In [ ]:
motif_drivers['Cardiomyocyte'].head(40)[motif_drivers['Cardiomyocyte'].head(40).index.isin(motif_drivers_de['Cardiomyocyte'])].index.to_list()

In [ ]:
gene_drivers_de['Cardiomyocyte'][~pd.Series(gene_drivers_de['Cardiomyocyte']).isin(gene_drivers['Cardiomyocyte'].head(40).index)]

In [ ]:
motif_drivers_de['Cardiomyocyte'][~pd.Series(motif_drivers_de['Cardiomyocyte']).isin(motif_drivers['Cardiomyocyte'].head(40).index)]

In [ ]:
motif_drivers['Epicardial'].head(40)[motif_drivers['Epicardial'].head(40).index.isin(motif_drivers_de['Epicardial'])].index.to_list()

In [ ]:
gene_drivers['Epicardial'].head(40)[gene_drivers['Epicardial'].head(40).index.isin(gene_drivers_de['Epicardial'])].shape

## Epicardial

In [ ]:
cluster = ['14']

In [ ]:
epi_drivers_motif = pd.read_csv(os.path.join(fig_path, f"jcf_epicardial_motif_drivers_{'_'.join(cluster)}.csv"), index_col=0)

In [ ]:
cluster = ['14', '12']

In [ ]:
epi_drivers_motif_2 = pd.read_csv(os.path.join(fig_path, f"jcf_epicardial_motif_drivers_{'_'.join(cluster)}.csv"), index_col=0)

In [ ]:
epi_drivers_motif.head(20).index[~epi_drivers_motif.head(20).index.isin(epi_drivers_motif_2.head(20).index)]

In [ ]:
# All the same

In [ ]:
cluster = ['14']

In [ ]:
drivers_motif = pd.read_csv(os.path.join(fig_path, f"jcf_cm_motif_drivers_{'_'.join(cluster)}.csv"), index_col=0)

In [ ]:
cluster = ['12', '14']

In [ ]:
drivers_motif_2 = pd.read_csv(os.path.join(fig_path, f"jcf_cm_motif_drivers_{'_'.join(cluster)}.csv"), index_col=0)

In [ ]:
drivers_motif_2.head(20).index[~drivers_motif_2.head(20).index.isin(drivers_motif.head(20).index)]

In [ ]:
cluster = ['14']

In [ ]:
drivers_motif = pd.read_csv(os.path.join(fig_path, f"jcf_epicardial_gene_drivers_{'_'.join(cluster)}.csv"), index_col=0)

In [ ]:
drivers_motif.head(100).index.isin(epi_drivers_motif.head(100).index)

In [ ]:
cluster = ['14']

In [ ]:
drivers_motif_2 = pd.read_csv(os.path.join(fig_path, f"jcf_cm_gene_drivers_{'_'.join(cluster)}.csv"), index_col=0)

In [ ]:
drivers_motif.head(100).index.isin(drivers_motif_2.head(100).index)

In [ ]:
drivers_motif_2.head(20).index[~drivers_motif_2.head(20).index.isin(drivers_motif.head(20).index)]

In [ ]:
drivers_motif.head(20).index[~drivers_motif.head(20).index.isin(drivers_motif_2.head(20).index)]